# Partie 1

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import linalg 
from scipy.spatial import distance

In [2]:
df_1 = pd.read_csv('notes.csv')
df_2= pd.read_csv('films.csv', encoding ='latin1')
df_2.head()
df = pd.merge(df_2, df_1, on = 'IdFilm')
df.head()

,IdFilm,Titre,Genres,IdUtilisateur,Note,Horodatage
0,1,Toy Story (1995),Aventure|Animation|Enfants|Comédie|Fantaisie,1,4.0,964982703
1,1,Toy Story (1995),Aventure|Animation|Enfants|Comédie|Fantaisie,5,4.0,847434962
2,1,Toy Story (1995),Aventure|Animation|Enfants|Comédie|Fantaisie,7,4.5,1106635946
3,1,Toy Story (1995),Aventure|Animation|Enfants|Comédie|Fantaisie,15,2.5,1510577970
4,1,Toy Story (1995),Aventure|Animation|Enfants|Comédie|Fantaisie,17,4.5,1305696483


In [3]:
# Delete duplicate rows and nan values  
print('la taille de notre jeu de données avant le pretraitement est: ',df.shape)
print('la taille de notre jeu de données apres la supprission de les lignes en double est: ',df.drop_duplicates().shape)
print('Le nombe de valeurs nan est: ', df.dropna().shape)

la taille de notre jeu de données avant le pretraitement est:  (100836, 6)
la taille de notre jeu de données apres la supprission de les lignes en double est:  (100836, 6)
Le nombe de valeurs nan est:  (100836, 6)


In [4]:
# 1-Quel est le nombre de notes donn´ees par tous les utilisateurs ?

In [5]:
print('Le nombre de notes données par tous les utilisateurs est: ',df.Note.notnull().shape[0])

Le nombre de notes données par tous les utilisateurs est:  100836


In [6]:
# 2-Quel est le nombre d’utilisateurs ? Quel est le nombre de films ?

In [7]:
print(" le nombre d’utilisateurs  est : ", df["IdUtilisateur"].nunique())
print(" le nombre de films est : ",  df[['IdFilm']].drop_duplicates().shape[0])

 le nombre d’utilisateurs  est :  610
 le nombre de films est :  9724


In [8]:
#3- Quel est le nombre moyen de notes par utilisateur ?

In [9]:
df_note= df.groupby(['IdUtilisateur']).size().reset_index(name='nb_moyen')
print('le nombre moyen de notes par utilisateur est: ',df_note.nb_moyen.mean())

le nombre moyen de notes par utilisateur est:  165.30491803278687


In [10]:
# 4-Quel est le nombre moyen de notes par film ?

In [11]:
df_note_film= df.groupby(['IdFilm']).size().reset_index(name='nb_film')
print('le nombre moyen de notes par film est: ',df_note_film.nb_film.mean() )

le nombre moyen de notes par film est:  10.369806663924312


In [12]:
# 5-Quel film a la note moyenne la plus basse?

In [13]:
df_min= df.groupby(['IdFilm', 'Titre'])['Note'].mean().reset_index(name='moyenne')
print(df_min[df_min['moyenne'] == df_min['moyenne'].min()])

      IdFilm                                              Titre  moyenne
2685    3604                                       Gypsy (1962)      0.5
2929    3933                          Killer Shrews, The (1959)      0.5
3023    4051  Horreurs of Spider Island (Ein Toter Hing im N...      0.5
3230    4371                                    Baby Boy (2001)      0.5
3364    4580                                      Cyborg (1989)      0.5
...      ...                                                ...      ...
9401  165645                                 Bad Santa 2 (2016)      0.5
9426  167296                                    Iron Man (1931)      0.5
9540  173307                     The Gracefield Incident (2015)      0.5
9572  175475                             The Emoji Movie (2017)      0.5
9675  184641                    Fullmetal Alchemist 2018 (2017)      0.5

[96 rows x 3 columns]


In [14]:
# 6. Quel film a la note moyenne la plus ´elev´ee ?

In [15]:
print(df_min[df_min['moyenne'] == df_min['moyenne'].max()])

      IdFilm                                              Titre  moyenne
48        53                                    Lamerica (1994)      5.0
87        99               Heidi Fleiss: Hollywood Madam (1995)      5.0
121      148                    Awfully Big Aventure, An (1995)      5.0
405      467                             Live Nude Girls (1995)      5.0
432      495  In the Realm of the Senses (Ai no corrida) (1976)      5.0
...      ...                                                ...      ...
9593  176601                                       Black Mirror      5.0
9615  179133                              Loving Vincent (2017)      5.0
9616  179135                              Blue Planet II (2017)      5.0
9670  184245                            De platte jungle (1978)      5.0
9693  187717                   Won't You Be My Neighbor? (2018)      5.0

[296 rows x 3 columns]


In [16]:
#df_note= df.groupby(['IdFilm','Note']).size().reset_index(name='nb_moyen')
#dd=df_note[['IdFilm','nb_moyen']].groupby("IdFilm").mean()
#print(dd[dd['nb_moyen'] == dd['nb_moyen'].min()])

In [17]:
# 1-a) Filtrez les utilisateurs qui ont vot´es moins de 100 films 

In [18]:
#df = pd.read_csv('notes.csv')
df2 = df.groupby('IdUtilisateur')['Note'].count()
filtering_Users= df2[df2 >=100].index

In [19]:
# 1-b)les films qui sont vot´es par moins de 10 utilisateurs? 

In [20]:
#df = pd.read_csv('notes.csv')
df3 = df.groupby('IdFilm')['Note'].count()
filtering_films= df3[df3>=10].index

In [21]:
#creation de matrice filtre

In [22]:
df_filtre = df[df["IdUtilisateur"].isin(list(filtering_Users))]
df_filtre = df_filtre[df_filtre["IdFilm"].isin(list(filtering_films))]
df_filtre.shape

(65964, 6)

In [23]:
# Construisez la matrice utilisateur-film? 

In [24]:
utilisateur_film = df_filtre.pivot(index = 'IdUtilisateur', columns='IdFilm', values='Note').fillna(0)
utilisateur_film.shape

(248, 2269)

In [25]:
# . Calculez le taux de parcimonie de la matrice utilisateur-film

In [26]:
A = np.count_nonzero(utilisateur_film)
B= utilisateur_film.count().sum()
s= A /B
print('taux de parcimonie de la matrice utilisateur-film est: ',s)

taux de parcimonie de la matrice utilisateur-film est:  0.11722515247586687


In [27]:
# Appliquez SVD sur la matrice utilisateur-film pour r´eduire la dimension 

In [28]:
U, s, Vh = linalg.svds(utilisateur_film, k = 15)

In [29]:
print('la taille de U est :', U.shape)
print('la taille de U est :', U.shape)
print('la taille de U est :', U.shape)

la taille de U est : (248, 15)
la taille de U est : (248, 15)
la taille de U est : (248, 15)


In [30]:
#2. Cherchez les 5 films les plus similaires au premier film ? Remarque: vous
#devez utiliser la fonction scipy.spatial.distance.cosine sur les lignes de la
#matrice V (apr`es la r´eduction de la dimension).

In [31]:
print( 'Le premier film est: ', df[df['IdFilm'] == 1]['Titre'][0])

Le premier film est:  Toy Story (1995)


In [32]:
distance.cosine([1, 0, 0], [0, 1, 0])
premier_film = Vh.T[0]
mesure = []
for i in range(1, 2269):
    m = distance.cosine(premier_film, Vh.T[i])
    mesure.append(m)

film_index = list(utilisateur_film.columns[1:])
print('Cherchez les 5 films les plus similaires au premier film est : \n ' )
sorted(zip(mesure, film_index))[-5:]


Cherchez les 5 films les plus similaires au premier film est : 
 


[(1.5429372672538793, 4487),
 (1.5497041143592614, 7160),
 (1.5589788599923309, 1779),
 (1.559797644463898, 7371),
 (1.5878422349363568, 46723)]

In [33]:
#3. Affichez les films recommend´es pour le premier utilisateur? 

In [34]:
M = np.dot(np.dot(U, np.diag(s)), Vh)
M = pd.DataFrame(M, columns=utilisateur_film.columns, index=utilisateur_film.index)

In [35]:
# acceder au premier index
film1= M.loc[1] 
#film1[film1>2.5].index

In [36]:
# les films que l utilisateur a notée plus que 3
list1 = list(film1[film1>3].index)

In [37]:
w = utilisateur_film.loc[1]
list2 = list(w[w != 0].index)

In [38]:
print('les films recommend´es pour le premier utilisateur et qui il n a pas encore regardé', [i for i in list1 if i not in list2])

les films recommend´es pour le premier utilisateur et qui il n a pas encore regardé [589, 1200, 1968, 2762, 2918]


In [39]:
liste_film_recommande = [i for i in list1 if i not in list2]
liste_film_recommande

[589, 1200, 1968, 2762, 2918]

In [40]:
film_recommande=df[df["IdFilm"].isin(list(liste_film_recommande))]
film_recommande.groupby(['IdFilm', 'Titre', 'Genres'])['IdFilm'].unique()

IdFilm  Titre                              Genres                        
589     Terminator 2: Judgment Day (1991)  Action|Sci-Fi                      [589]
1200    Aliens (1986)                      Action|Aventure|Horreur|Sci-Fi    [1200]
1968    Breakfast Club, The (1985)         Comédie|Drame                     [1968]
2762    Sixth Sense, The (1999)            Drame|Horreur|Mystère             [2762]
2918    Ferris Bueller's Day Off (1986)    Comédie                           [2918]
Name: IdFilm, dtype: object